In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
import h5py
from keras.models import load_model

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

In [3]:
train_data_path = '/home/ec2-user/Capstone_Project/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'
test_data_path = '/home/ec2-user/Capstone_Project/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

In [4]:
# Read data form training and test 

In [5]:
# X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
#                                              masks_path = train_mask_path, 
#                                              crop_size = 33, 
#                                              num_classes = 6, 
#                                              samples = 100000, 
#                                              balance = True)

In [6]:
# X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
#                                            masks_path = test_mask_path, 
#                                            crop_size = 33, 
#                                            num_classes = 6, 
#                                            samples = 10000, 
#                                            balance = True)

In [7]:
# print(type(X_train), type(Y_train))
# print(X_train.shape, Y_train.shape, num_bands)
# print(X_test.shape,  Y_test.shape,  num_bands)

In [8]:
# X_train_list = np.array([X_train[i] for i in range(X_train.shape[0])])

In [9]:
# X_test_list = np.array([X_test[i] for i in range(X_test.shape[0])])

In [10]:
# Y_train_list = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [11]:
# Y_test_list = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Batch Generator

This function create a generator for keras.  

In [4]:
def DataGenerator(data_path, masks_path, crop_size, num_classes, samples = 1000, balance = True):  
    while True:
        X, Y, num_bands = utils.load_data(data_path, 
                                          masks_path, 
                                          crop_size, 
                                          num_classes, 
                                          samples,
                                          balance)
        X = np.array([X[i] for i in range(X.shape[0])])
        Y = np.array([Y[i,:] for i in range(Y.shape[0])])
        yield X,Y

# Build First Model

In [13]:
# model = Sequential()
# model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1),
#                  activation='relu',
#                  input_shape=(33, 33, 16),
#                  kernel_regularizer=l2(0.01)))

# model.add(MaxPooling2D(pool_size=(2, 2),
#                        strides=(2, 2)))

# model.add(Conv2D(filters=32, kernel_size=(5, 5),
#                  strides=(1, 1),
#                  activation='relu',
#                  kernel_regularizer=l2(0.01)))

# model.add(MaxPooling2D(pool_size=(2, 2),
#                        strides=(2, 2)))
# model.add(Flatten())
# model.add(Dense(128,
#                 activation='relu',
#                 kernel_regularizer=l2(0.01)))
# model.add(Dense(6,
#                 activation='softmax',
#                 kernel_regularizer=l2(0.01)))

In [5]:
X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
                                           masks_path = test_mask_path, 
                                           crop_size = 33, 
                                           num_classes = 6, 
                                           samples = 1000, 
                                           balance = True)

In [6]:
X_test = np.array([X_test[i] for i in range(X_test.shape[0])])

In [7]:
Y_test = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

In [8]:
train_generator = DataGenerator(train_data_path,
                                train_mask_path, 
                                crop_size = 33, 
                                num_classes = 6, 
                                samples = 1000, 
                                balance = True)

In [9]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=(1, 1), input_shape=(33, 33, 16), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, kernel_regularizer=l2(0.01)))
model.add(Activation('softmax'))             

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [11]:
model.fit_generator(generator = train_generator,
                    samples_per_epoch = 10,
                    epochs = 10,
                    validation_steps = 1,
                    validation_data = (X_test,Y_test))

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=10, validation_steps=1, validation_data=(array([[[..., steps_per_epoch=10)`
  """


Epoch 1/10
 6/10 [=================>............] - ETA: 3:31 - loss: 5.9378 - acc: 0.2774

KeyboardInterrupt: 

In [27]:
model.save('model_2_1.h5')

In [3]:
model = load_model('model_2_1.h5')

In [1]:
import classify